<a href="https://colab.research.google.com/github/panthamramesh/Qualcomm-DL-Hackathon/blob/master/DL_Hackathon_Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 29.78 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
import os
import zipfile

In [4]:
import zipfile
import os
import shutil

def extract_files_only(zip_file_paths, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for zip_file_path in zip_file_paths:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for member in zip_ref.namelist():
                # Check if the member is a file (not a directory)
                if not member.endswith('/'):
                    # Extract the file to a temporary location
                    source = zip_ref.open(member)
                    target_path = os.path.join(destination_folder, os.path.basename(member))
                    with open(target_path, "wb") as target:
                        shutil.copyfileobj(source, target)

# Example usage
zip_file_paths = ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip']
destination_folder = '/content/Qualcomm-DL-Hackathon/Images'

extract_files_only(zip_file_paths, destination_folder)
print(f"Files from {zip_file_paths} have been extracted to {destination_folder}.")

Files from ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip'] have been extracted to /content/Qualcomm-DL-Hackathon/Images.


In [5]:
def count_files_in_folder(folder_path):
    with os.scandir(folder_path) as entries:
        return sum(1 for entry in entries if entry.is_file())

# Example usage
folder_path = '/content/Qualcomm-DL-Hackathon/Images'
file_count = count_files_in_folder(folder_path)
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 2352


In [6]:
import os
import pandas as pd
from PIL import Image
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset,TensorDataset,random_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [7]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# Load the train and test CSV files
train_df = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
test_df = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
def load_data(df, root_dir, transform):
    images = []
    labels = []
    for idx, row in df.iterrows():
        img_name = row['image_names']
        img_path = os.path.join(root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        images.append(image)
        if 'emergency_or_not' in df.columns:
            labels.append(row['emergency_or_not'])
    return torch.stack(images), torch.tensor(labels)

train_images, train_labels = load_data(train_df, '/content/Qualcomm-DL-Hackathon/Images', transform)
test_images, _ = load_data(test_df, '/content/Qualcomm-DL-Hackathon/Images', transform)

dataset = TensorDataset(train_images, train_labels)

# Split the dataset into training and validation sets (80% train, 20% val)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [11]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)

In [13]:
num_epochs = 10
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation phase
    model.eval()
    val_corrects = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)

    val_accuracy = val_corrects.double() / len(val_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Save the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_vehicle_classification_model.pth')

Epoch [1/10], Loss: 0.4374, Val Accuracy: 0.8121
Epoch [2/10], Loss: 0.2126, Val Accuracy: 0.9030
Epoch [3/10], Loss: 0.1376, Val Accuracy: 0.8212
Epoch [4/10], Loss: 0.1764, Val Accuracy: 0.7364
Epoch [5/10], Loss: 0.0902, Val Accuracy: 0.8758
Epoch [6/10], Loss: 0.1145, Val Accuracy: 0.9000
Epoch [7/10], Loss: 0.1414, Val Accuracy: 0.7909
Epoch [8/10], Loss: 0.1063, Val Accuracy: 0.8970
Epoch [9/10], Loss: 0.0302, Val Accuracy: 0.8667
Epoch [10/10], Loss: 0.0585, Val Accuracy: 0.7485


In [16]:
model.load_state_dict(torch.load('best_vehicle_classification_model.pth'))
model.eval()

<ipython-input-16-28248372f4a2>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_vehicle_classification_model.pth'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
test_dataset = TensorDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
predictions = []

with torch.no_grad():
    for images in test_loader:
        outputs = model(images[0])
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

# Add predictions to the test dataframe and save to CSV
test_df['emergency_or_not'] = predictions
test_df.to_csv('test_predictions.csv', index=False)

print("Predictions saved to test_predictions.csv")

Predictions saved to test_predictions.csv
